In [ ]:
import pandas as pd
import plotly.express as px
import requests
import json
import datetime
import concurrent.futures
import swifter
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from numpy import random

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
if os.path.isfile('dados/TJRN_4º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2018-01-01.csv'):
    df_tjrn = pd.read_csv('dados/TJRN_4º_JUIZADO_ESPECIAL_DA_FAZENDA_PÚBLICA_2018-01-01.csv', header='infer', sep=';', compression='zip')
else:
    df_tjrn = pd.read_csv(glob.glob("dados/TJRN_17*.csv")[0], header='infer', sep=';', compression='zip')
    print(f'Carregando o dataset referente ao {glob.glob("dados/TJRN_17*.csv")[0]}')
    

In [ ]:
df_tjrn.drop_duplicates(inplace=True)
if 'sort' in df_tjrn.columns:
    df_tjrn.drop(columns=['sort'], inplace=True)
if 'classe' in df_tjrn.columns:
    df_tjrn.drop(columns=['classe'], inplace=True)    
if 'grau' in df_tjrn.columns:
    df_tjrn.drop(columns=['grau'], inplace=True)
if 'orgao_julgador' in df_tjrn.columns:
    df_tjrn.drop(columns=['orgao_julgador'], inplace=True)
    
if df_tjrn['assuntos'].isna().sum() > (0.8 * len(df_tjrn['assuntos'])):
    df_tjrn.drop(columns=['assuntos'], inplace=True)    

In [ ]:
df_tjrn.head(1)

In [ ]:

df_tjrn['data_ajuizamento'] = pd.to_datetime(df_tjrn['data_ajuizamento'], utc=True)
print(df_tjrn.info())
df_tjrn['data_ajuizamento'] = df_tjrn['data_ajuizamento'].dt.strftime('%Y-%m-%d')
df_tjrn['ultima_atualizacao'] = pd.to_datetime(df_tjrn['ultima_atualizacao'], format='mixed').dt.strftime('%Y-%m-%d')
print(df_tjrn.head(2))

In [ ]:
movimentacoes = list(df_tjrn.movimentos.values)
print(len(movimentacoes))

In [ ]:
print(movimentacoes[0])
print(df_tjrn['movimentos'][0])

In [ ]:
""" lista_movimentacoes_completa = []
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        continue 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                if elemento != None:
                    codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                    nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                    data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                    lista_temp = list()
                    lista_temp.append([codigo, nome_movimentacao, data_movimentacao])
                    #lista_temp.append(nome_movimentacao)
                    #lista_temp.append(data_movimentacao)
                    #lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa.append(lista_temp)
                else:
                    lista_temp = [-1, -1, -1]
                    #lista_movimentacoes_completa.append([])
                    lista_movimentacoes_completa.append(lista_temp)
            i = i + 1 """

In [ ]:
lista_movimentacoes_completa = []
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        #continue 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            lista_temp = list()
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                lista_temp.append([codigo, nome_movimentacao, data_movimentacao])
        else:
            lista_temp.append([-1,-1,-1])
    lista_movimentacoes_completa.append(lista_temp)


In [ ]:
print(len(lista_movimentacoes_completa))
#lista_movimentacoes_completa = list(filter(None, lista_movimentacoes_completa))
print(len(movimentacoes), '\t', movimentacoes[0])
print(len(lista_movimentacoes_completa), '\t', lista_movimentacoes_completa[0])


In [ ]:
print(type(lista_movimentacoes_completa), len(lista_movimentacoes_completa))
print(type(lista_movimentacoes_completa[0]), len(lista_movimentacoes_completa[2]))
print(lista_movimentacoes_completa[5200])
print(lista_movimentacoes_completa[:10])

In [ ]:
print(lista_movimentacoes_completa[0])
print(df_tjrn['movimentos'][0])

In [ ]:
movimentacoes_gabinete = pd.read_csv('dados/movimentos_gabinete.csv', sep=';', header='infer')
movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
movimentacoes_gabinete.drop(columns=['complemento','vazio', 'vazio.2', 'vazio.1'], inplace=True)
lista_movimentacoes_sentenca = movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento']['codigo'].to_list()
print(movimentacoes_gabinete.head(5))

In [ ]:
print(movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento'].head(5))

In [ ]:
movimentacoes_secretaria = pd.read_csv('dados/movimentos_secretaria.csv', sep=';', header='infer')
print(movimentacoes_secretaria.head(5))
movimentacoes_secretaria.drop(columns=['Unnamed: 3','complemento'], inplace=True)
lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()
print(movimentacoes_secretaria.head(5))

In [ ]:
def calcular_data_sentenca(movimentacoes):
    processos_sentenciados = 0
    processos_nao_sentenciados = 0    
    lista_data_sentenca = list()
    for i in range(0,(len(movimentacoes)),1):
        processado = False
        if len(movimentacoes[i]) > 3:
            for j in range(0,(len(movimentacoes[i])-1),3):
                #print(movimentacoes[i][j][0])
                if processado:
                   break 
                if int(movimentacoes[i][j][0]) in lista_movimentacoes_sentenca:
                    data = pd.to_datetime(movimentacoes[i][j][2])
                    data = data.date().strftime("%Y-%m-%d")
                    lista_data_sentenca.append(data)
                    processos_sentenciados = processos_sentenciados + 1
                    processado = True
                    #print(f'Processando: {i}: Sentença encontrada')
            if not processado:
                lista_data_sentenca.append('-1')
                processado = True
                processos_nao_sentenciados = processos_nao_sentenciados + 1         
   
                #print(f'data adicionada: {data}')    
        else:
            #print(movimentacoes[i])
            lista_data_sentenca.append('-1')
            processos_nao_sentenciados = processos_nao_sentenciados + 1
            #print(f'data adicionada: {data_default_nao_sentenciados}')
            #print(f'Processando: {i}: SEM movimentação do gabinete')
    print(f'processos sentenciados: {processos_sentenciados}\tprocessos não sentenciados: {processos_nao_sentenciados}')
    return lista_data_sentenca

In [ ]:
#print(calcular_data_sentenca(lista_movimentacoes_completa))
print(len(lista_movimentacoes_completa), len(calcular_data_sentenca(lista_movimentacoes_completa)))

In [ ]:
# funciona ok.
def calcular_tempo_entre_movimentacoes(movimentacoes, mov_inicial, mov_final):
    lista_duracao = list()
    for i in range(0,(len(movimentacoes)),1):
        #if i < 3:
            #print(movimentacoes[i])
        if ((len(movimentacoes[i]) > 3) and (len(movimentacoes[i]) >= mov_final)):
            try:
                duracao = abs(pd.to_datetime(movimentacoes[i][mov_final][2]) - pd.to_datetime(movimentacoes[i][mov_inicial][2]))/np.timedelta64(1, 'D')
                lista_duracao.append(int(duracao))
            except:
                lista_duracao.append(int(-1))          

        else: 
            lista_duracao.append(int(-1))
            
    return lista_duracao        

In [ ]:
df_tjrn['data_sentenca'] = calcular_data_sentenca(lista_movimentacoes_completa)
df_tjrn['movimentos'] = lista_movimentacoes_completa

In [ ]:
df_tjrn['tempo_entre_1e2_mov'] = calcular_tempo_entre_movimentacoes(df_tjrn['movimentos'].to_list(),0,1)
df_tjrn['tempo_entre_2e3_mov'] = calcular_tempo_entre_movimentacoes(df_tjrn['movimentos'].to_list(),1,2)
df_tjrn['tempo_entre_3e4_mov'] = calcular_tempo_entre_movimentacoes(df_tjrn['movimentos'].to_list(),2,3)
#df_tjrn['tempo_entre_4e5_mov'] = calcular_tempo_entre_movimentacoes(df_tjrn['movimentos'].to_list(),4,5)

df_tjrn['mais60d'] = (df_tjrn['tempo_entre_1e2_mov'] > 60) | (df_tjrn['tempo_entre_2e3_mov'] > 60) | (df_tjrn['tempo_entre_3e4_mov'] > 60) #| (df_tjrn['tempo_entre_4e5_mov'] > 60)

In [ ]:
print(df_tjrn['mais60d'].value_counts())

In [ ]:
df_sentenciados = df_tjrn[df_tjrn['data_sentenca'] != '-1']
df_sentenciados['tempo_ate_sentenca'] = abs(pd.to_datetime(df_sentenciados['data_sentenca']) - pd.to_datetime(df_sentenciados['data_ajuizamento']))/np.timedelta64(1, 'D')
df_nao_sentenciados = df_tjrn[df_tjrn['data_sentenca'] == '-1']
df_nao_sentenciados['tempo_ate_sentenca'] = 0

In [ ]:
print(df_sentenciados.shape)
print(df_nao_sentenciados.shape)
print(df_tjrn.shape)

In [ ]:
df_tjrn = pd.concat([df_sentenciados, df_nao_sentenciados])
print(df_tjrn.head(1))

## Estatísticas iniciais

In [ ]:
#print(df_sentenciados.shape[0], df_nao_sentenciados.shape[0])
percentual_sentenciados = (int(df_sentenciados.shape[0])/int(df_nao_sentenciados.shape[0]) * 100)
percentual_sentenciados = float("{:.4f}".format(percentual_sentenciados))
print(f'Percentual da base de casos sentenciados: {percentual_sentenciados}%')

In [ ]:
df_sentenciados.isna().sum()

In [ ]:
df_sentenciados.info()

In [ ]:
fig = px.histogram(df_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos já sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_nao_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos ainda não sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_sentenciados, x='tempo_ate_sentenca', marginal='rug', title='Distribuição do tempo até a sentença')
fig.show()

## Testes com ML

In [ ]:
print(df_tjrn.columns)

In [ ]:
df_ml = df_sentenciados.drop(columns=['movimentos', 'numero_processo', 'data_ajuizamento', 'ultima_atualizacao', 'codigo', 'data_sentenca' ])
df_ml = pd.get_dummies(df_ml, columns=['assuntos'], drop_first=True)
#print(df_ml.head(1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

metricas = ['accuracy', 'f1', 'precision', 'recall']
rfc = RandomForestClassifier(random_state=19, n_jobs=-1)
adc = AdaBoostClassifier(random_state=19,algorithm="SAMME", n_estimators=200)
mlp = MLPClassifier(random_state=19)
dtc = DecisionTreeClassifier(random_state=19)

scores_rfc = cross_validate(rfc, (df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])),
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_adc = cross_validate(adc, (df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])),
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_mlp = cross_validate(mlp, (df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])),
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_dtc = cross_validate(dtc, (df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])),
                             df_ml['mais60d'], cv=10, scoring=metricas)

In [ ]:
chaves = scores_dtc.keys()

print(f'Métrica \tRandomForest \tAdaBoost \tMLP \tDecisionTree')
for chave in chaves:
    print(f'{chave}:\t {scores_rfc[chave].mean():.3f} \t\t{scores_adc[chave].mean():.3f} \t\t{scores_mlp[chave].mean():.3f} \t\t{scores_dtc[chave].mean():.3f}')


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor


from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split((df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d'])), 
                                                    df_ml['mais60d'], test_size=0.20, random_state=42)
model = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=250)
model.fit(X_train, y_train)
rf_y_pred = model.predict(X_test)

rf_r2 = r2_score(y_test, rf_y_pred)
rf_mae = mean_absolute_error(y_test, rf_y_pred)
rf_rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
rf_mse = mean_squared_error(y_test, rf_y_pred)

print('----------------------------------------------------')

print(f'AdaBoost Regressor')
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)
ada_y_pred = regr.predict(X_test)

ada_r2 = r2_score(y_test, ada_y_pred)
ada_mae = mean_absolute_error(y_test, ada_y_pred)
ada_rmse = mean_squared_error(y_test, ada_y_pred, squared=False)
ada_mse = mean_squared_error(y_test, ada_y_pred)

print('----------------------------------------------------')

print(f'MLP Regressor')

regr_mlp = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
mlp_y_pred = regr_mlp.predict(X_test)

mlp_r2 = r2_score(y_test, mlp_y_pred)
mlp_mae = mean_absolute_error(y_test, mlp_y_pred)
mlp_rmse = mean_squared_error(y_test, mlp_y_pred, squared=False)
mlp_mse = mean_squared_error(y_test, mlp_y_pred)

print('----------------------------------------------------')

print(f'DT Regressor')
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
dt_y_pred = regr_mlp.predict(X_test)

dt_r2 = r2_score(y_test, dt_y_pred)
dt_mae = mean_absolute_error(y_test, dt_y_pred)
dt_rmse = mean_squared_error(y_test, dt_y_pred, squared=False)
dt_mse = mean_squared_error(y_test, dt_y_pred)

print(f'Metric\t\t\t\t RF\t AdaBoost\t MLP\t DT')
print(f"Mean Squared Error:\t\t {rf_r2:.3f}\t {ada_r2:.3f}\t {mlp_r2:.3f}\t {dt_r2:.3f}")
print(f"R-squared (R²):\t\t {rf_mae:.3f}\t {ada_mae:.3f}\t {mlp_mae:.3f}\t {dt_mae:.3f}")
print(f"Mean Absolute Error (MAE):\t\t {rf_mae:.3f}\t {ada_rmse:.3f}\t {mlp_rmse:.3f}\t {dt_rmse:.3f}")
print(f"Root Mean Squared Error (RMSE):\t\t {rf_rmse:.3f}\t {ada_mse:.3f}\t {mlp_mse:.3f}\t {dt_mse:.3f}")